In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import zipfile


zip_ref = zipfile.ZipFile("/content/drive/My Drive/Colab Notebooks/MURA-v1.1.zip", 'r')
zip_ref.extractall("/tmp/SML")
zip_ref.close()

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm
import cv2
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import os
from keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
np.random.seed(1)

In [0]:
df = pd.read_csv("/tmp/SML/MURA-v1.1/train_image_paths.csv",
                 names=["x_col"])

df['Region'] = df["x_col"].map(lambda x: x.split('/')[-4])
df["Region"] = df["Region"].map(lambda x: x.split('XR_')[-1])
df["y_col"] = df["x_col"].map(lambda x: x.split('/')[-2])
df["y_col"] = df["y_col"].map(lambda x: x.split('_')[-1])
df

In [0]:
X = df["x_col"]
y = df["Region"]

In [0]:
df_test = pd.read_csv("/tmp/SML/MURA-v1.1/valid_image_paths.csv",
                 names=["x_col"])
df_test['Region'] = df_test["x_col"].map(lambda x: x.split('/')[-4])
df_test["Region"] = df_test["Region"].map(lambda x: x.split('XR_')[-1])
df_test["y_col"] = df_test["x_col"].map(lambda x: x.split('/')[-2])
df_test["y_col"] = df_test["y_col"].map(lambda x: x.split('_')[-1])
df_test

In [0]:
X_train_path, X_test_path, y_train, y_test = train_test_split(X, y, test_size = 0.20,stratify=y, random_state = 57)

In [0]:
df_train=pd.DataFrame()
df_valid = pd.DataFrame()
df_train["x_col"]=X_train_path
df_train["y_col"]=y_train
df_valid["x_col"]=X_test_path
df_valid["y_col"]=y_test

In [0]:
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.optimizers import Adam
from keras.models import Model
# DATASET_PATH  = '/tmp/SML/MURA-v1.1/train/WRIST'
IMAGE_SIZE    = (224, 224)
NUM_CLASSES   = 1
BATCH_SIZE    = 16  # try reducing batch size or freeze more layers if your GPU runs out of memory
FREEZE_LAYERS = 2  # freeze the first this many layers for training
NUM_EPOCHS    = 15
WEIGHTS_FINAL = 'model-vgg16-abnormal.h5'


train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,
                                   horizontal_flip=True)

train_batches = train_datagen.flow_from_dataframe(
        dataframe=df_train,
        directory='/tmp/SML',
        x_col="x_col",
        y_col="y_col",
        interpolation='bicubic',
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=True,
        seed =57)


# train_batches = train_datagen.flow_from_directory(DATASET_PATH + '/train',
#                                                   target_size=IMAGE_SIZE,
#                                                   interpolation='bicubic',
#                                                   class_mode='binary',
#                                                   shuffle=True,
#                                                   batch_size=BATCH_SIZE)

valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

valid_batches = valid_datagen.flow_from_dataframe(
        dataframe=df_valid,
        directory='/tmp/SML',
        x_col="x_col",
        y_col="y_col",
        target_size=IMAGE_SIZE,
        interpolation='bicubic',
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        seed = 57)

# valid_batches = valid_datagen.flow_from_directory(DATASET_PATH + '/valid',
#                                                   target_size=IMAGE_SIZE,
#                                                   interpolation='bicubic',
#                                                   class_mode='binary',
#                                                   shuffle=False,
#                                                   batch_size=BATCH_SIZE)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_batches=test_datagen.flow_from_dataframe(
dataframe=df_test,
directory="/tmp/SML",
x_col="x_col",
y_col=None,
batch_size=1,
seed=57,
shuffle=False,
class_mode=None,
target_size=IMAGE_SIZE)

In [0]:
import keras
from keras.models import Model, load_model
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16


#Load the VGG model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

#print(base_model.summary())

    # Freeze the layers 
for layer in base_model.layers:
    layer.trainable = False
 
# # Create the model
model = keras.models.Sequential()

# # Add the vgg convolutional base model
model.add(base_model)
 
# # Add new layers
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
 
# # Show a summary of the model. Check the number of trainable parameters    
#print(model.summary())

In [0]:
from keras.optimizers import SGD

model.compile(loss='binary_crossentropy',
          optimizer=SGD(lr=1e-3),
          metrics=['accuracy'])

# # Start the training process
# model.fit(x_train, y_train, validation_split=0.30, batch_size=32, epochs=50, verbose=2)

history = model.fit_generator(
        train_batches,
        steps_per_epoch=train_batches.n/batch_size,
        validation_data = valid_batches,
        validation_steps = valid_batches.n/batch_size,
        epochs=20)

In [0]:
score =  model.evaluate_generator(valid_generator,steps=valid_generator.n/batch_size)
score